### Домашнее задание

Нужно реализовать rest api на базе flask (пример https://github.com/fimochka-sudo/GB_docker_flask_example)

По шагам:
0. выбрать себе датасет (который интересен или нравится больше всего), сделать pipeline (преобразования + модель), сохранить его на диск. Если не хочется пайплайн, то можно без него, но так вам же будет удобнее потом вызывать его из кода сервиса.
1. установить удобную для себя среду разработки (pycharm прекрасен - https://www.jetbrains.com/pycharm/)
2. для вашего проекта вам понадобится requirements.txt с пакетами. Можно за основу взять такой файл из проекта выше. Для его установки прям в pycharm можно открыть терминал и сделать pip install -r requirements.txt (находясь в корне проекта конечно же при этом)
3. завести себе аккаунт на github (если его еще нет). У самого github есть такой "hello world" по работе с ним - https://guides.github.com/activities/hello-world/
4. итоговый проект должен содержать: 1) каталог app/models/ (здесь модель-пайплайн предобученная либо код обучения модели-пайплайна) 2) файл app/run_server.py (здесь основной код flask-приложения) 3) requirements.txt (список пакетов, которые у вас используются в проекте - в корне проекта) 4) README.md (здесь какое-то описание, что вы делаете, что за данные, как запускать и т.д) 5) Dockerfile 6) docker-entrypoint.sh
5. (<b>Опционально</b>): front-end сервис какой-то, который умеет принимать от пользователя введеные данные и ходить в ваш api. На самом деле полезно больше вам, т.к если ваш проект будет далее развиваться (новые модели, интересные подходы), то это хороший пунктик к резюме и в принципе - строчка в портфолио)

Полезные ссылки:
1. датасеты (для полета мысли): https://www.kaggle.com/datasets
2. конкурс Сбербанка по недвижимости (можно этот набор данных также взять и обучить модель предсказывать стоимость жилья - неплохой такой сервис может получиться) - https://www.kaggle.com/c/sberbank-russian-housing-market/data Там же и ноутбуки с разными подходами есть.
3. минималистичный пример связки keras/flask https://blog.keras.io/building-a-simple-keras-deep-learning-rest-api.html для определения класса картинки
4. неплохой такой пример (помимо того, что разобрали на занятии) связки docker/flask - https://cloud.croc.ru/blog/byt-v-teme/flask-prilozheniya-v-docker/
5. https://www.digitalocean.com/community/tutorials/how-to-build-and-deploy-a-flask-application-using-docker-on-ubuntu-18-04

p.s. если проблемы с выбором датасета, то пишите пожалуйста - будем вместе думать)

Cоздать модель машинного обучения, которая предсказывает, какие твиты посвящены реальным бедствиям, а какие нет.

https://www.kaggle.com/code/philculliton/nlp-getting-started-tutorial/input

#### Столбцы

- id — уникальный идентификатор для каждого твита
- text - текст твита
- location — место, откуда был отправлен твит (может быть пустым)
- keyword - конкретное ключевое слово из твита (может быть пустым)
- target — только в train.csv, указывает, относится ли твит к настоящей катастрофе (1) или нет (0)

In [1]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
import dill

import matplotlib.pyplot as plt
# расширить колонки вывода
pd.set_option('display.max_colwidth', None)

In [2]:
df = pd.read_csv('train.csv')
df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are being notified by officers. No other evacuation or shelter in place orders are expected,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation orders in California",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as smoke from #wildfires pours into a school,1


In [3]:
df.shape

(7613, 5)

In [4]:
df.loc[~df['keyword'].isnull()].iloc[:10]

,id,keyword,location,text,target
31,48,ablaze,Birmingham,@bbcmtd Wholesale Markets ablaze http://t.co/lHYXEOHY6C,1
32,49,ablaze,Est. September 2012 - Bristol,We always try to bring the heavy. #metal #RT http://t.co/YAo1e0xngw,0
33,50,ablaze,AFRICA,#AFRICANBAZE: Breaking news:Nigeria flag set ablaze in Aba. http://t.co/2nndBGwyEi,1
34,52,ablaze,"Philadelphia, PA",Crying out for more! Set me ablaze,0
35,53,ablaze,"London, UK",On plus side LOOK AT THE SKY LAST NIGHT IT WAS ABLAZE http://t.co/qqsmshaJ3N,0
36,54,ablaze,Pretoria,@PhDSquares #mufc they've built so much hype around new acquisitions but I doubt they will set the EPL ablaze this season.,0
37,55,ablaze,World Wide!!,INEC Office in Abia Set Ablaze - http://t.co/3ImaomknnA,1
38,56,ablaze,NaN,Barbados #Bridgetown JAMAICA ÛÒ Two cars set ablaze: SANTA CRUZ ÛÓ Head of the St Elizabeth Police Superintende... http://t.co/wDUEaj8Q4J,1
39,57,ablaze,Paranaque City,Ablaze for you Lord :D,0
40,59,ablaze,Live On Webcam,Check these out: http://t.co/rOI2NSmEJJ http://t.co/3Tj8ZjiN21 http://t.co/YDUiXEfIpE http://t.co/LxTjc87KLS #nsfw,0


In [5]:
df['keyword'].isnull().sum()

61

In [6]:
df['location'].isnull().sum()

2533

In [7]:
df['text'].isnull().sum()

0

In [8]:
df.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [9]:
df.loc[df['keyword'].isnull()].iloc[:10]

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are being notified by officers. No other evacuation or shelter in place orders are expected,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation orders in California",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as smoke from #wildfires pours into a school,1
5,8,NaN,NaN,#RockyFire Update => California Hwy. 20 closed in both directions due to Lake County fire - #CAfire #wildfires,1
6,10,NaN,NaN,"#flood #disaster Heavy rain causes flash flooding of streets in Manitou, Colorado Springs areas",1
7,13,NaN,NaN,I'm on top of the hill and I can see a fire in the woods...,1
8,14,NaN,NaN,There's an emergency evacuation happening now in the building across the street,1
9,15,NaN,NaN,I'm afraid that the tornado is coming to our area...,1


In [10]:
class DataPreprocessing():
    """Подготовка исходных данных"""
    
    def __init__(self, key):
        """Параметры класса"""
        self.key = key
        self.indexer = None
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        """Обработка выбросов и пропусков"""
        self.indexer = X.loc[X[self.key].isna()].index
        X.drop(self.indexer, inplace=True)
        return X

In [11]:
# Удалим пропуски
data_pre = DataPreprocessing(key='keyword')
df = data_pre.transform(df)

In [12]:
# разделим данные на train/test
X_train, X_test, y_train, y_test = train_test_split(df, df['target'], random_state=42)

# Сохраним тест
X_test.to_csv("X_test.csv", index=None)
y_test.to_csv("y_test.csv", index=None)

# Сохраним трайн
X_train.to_csv("X_train.csv", index=None)
y_train.to_csv("y_train.csv", index=None)

In [13]:
X_train.shape, y_train.shape

((5664, 5), (5664,))

In [14]:
# соберем наш простой pipeline, но нам понадобится написать класс для выбора нужного поля
    
class ColumnSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        return X[self.key]
    
class OHEEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key
        self.columns = []

    def fit(self, X, y=None):
        self.columns = [col for col in pd.get_dummies(X, prefix=self.key).columns]
        return self

    def transform(self, X):
        X = pd.get_dummies(X, prefix=self.key)
        test_columns = [col for col in X.columns]
        for col_ in test_columns:
            if col_ not in self.columns:
                X[col_] = 0
        return X.reindex(columns=self.columns).fillna(0)
    
keyword = Pipeline([
     ('selector', ColumnSelector('keyword')),
     ('ohe', OHEEncoder('keyword'))

])

tfidf = Pipeline([
     ('selector', ColumnSelector('text')), 
     ('tfidf', TfidfVectorizer(stop_words='english'))
])

from sklearn.pipeline import FeatureUnion
feats = FeatureUnion([('keyword', keyword),
                      ('tfidf', tfidf)
                     ])

pipeline = Pipeline([
     ('feats', feats),
     #('clf', DecisionTreeClassifier(random_state=42)),
     ('clf', LogisticRegression())
])

In [15]:
# Запустим кросс-валидацию LogisticRegression
param_grid = {'feats__tfidf__tfidf__max_features': range(100, 2000, 100),
              'feats__tfidf__tfidf__max_df': (0.5, 0.7, 1),
              'clf__penalty' : ['l2'],
              'clf__C': (0.01, 0.1, 1)
             }
gs_cv = GridSearchCV(pipeline, param_grid, cv=3, n_jobs=8, scoring='roc_auc')

In [16]:
%%time
gs_cv.fit(X_train, y_train)

CPU times: user 4.89 s, sys: 955 ms, total: 5.85 s
Wall time: 11.1 s


GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('feats',
                                        FeatureUnion(transformer_list=[('keyword',
                                                                        Pipeline(steps=[('selector',
                                                                                         ColumnSelector(key='keyword')),
                                                                                        ('ohe',
                                                                                         OHEEncoder(key='keyword'))])),
                                                                       ('tfidf',
                                                                        Pipeline(steps=[('selector',
                                                                                         ColumnSelector(key='text')),
                                                                                        ('tfidf',
             

In [17]:
print(gs_cv.best_score_)
print(gs_cv.best_params_)

0.8406105166124228
{'clf__C': 1, 'clf__penalty': 'l2', 'feats__tfidf__tfidf__max_df': 0.5, 'feats__tfidf__tfidf__max_features': 1300}


In [18]:
# Построим новый пайплайн
keyword = Pipeline([
     ('selector', ColumnSelector('keyword')),
     ('ohe', OHEEncoder('keyword'))

])

tfidf = Pipeline([
     ('selector', ColumnSelector('text')), 
     ('tfidf', TfidfVectorizer(max_features=1300, max_df=0.5, stop_words='english'))
])

from sklearn.pipeline import FeatureUnion
feats = FeatureUnion([('keyword', keyword),
                      ('tfidf', tfidf)
                     ])

pipeline = Pipeline([
     ('feats', feats),
     ('clf', LogisticRegression())
])

In [19]:
pipeline.fit(X_train, y_train)

Pipeline(steps=[('feats',
                 FeatureUnion(transformer_list=[('keyword',
                                                 Pipeline(steps=[('selector',
                                                                  ColumnSelector(key='keyword')),
                                                                 ('ohe',
                                                                  OHEEncoder(key='keyword'))])),
                                                ('tfidf',
                                                 Pipeline(steps=[('selector',
                                                                  ColumnSelector(key='text')),
                                                                 ('tfidf',
                                                                  TfidfVectorizer(max_df=0.5,
                                                                                  max_features=1300,
                                                                   

In [20]:
preds = pipeline.predict_proba(X_test)[:, 1]
preds

array([0.88816423, 0.59430322, 0.0871772 , ..., 0.11298493, 0.81759869,
       0.37896838])

In [21]:
from sklearn.metrics import f1_score, roc_auc_score, precision_score, classification_report, precision_recall_curve
b=1
precision, recall, thresholds = precision_recall_curve(y_test.values, preds)
fscore = (1+b**2)*(precision * recall) / (b**2*precision + recall)
# locate the index of the largest f score
ix = np.argmax(fscore)
print('Best Threshold=%f, F-Score=%.3f, Precision=%.3f, Recall=%.3f' % (thresholds[ix], 
                                                                        fscore[ix],
                                                                        precision[ix],
                                                                        recall[ix]))

Best Threshold=0.493727, F-Score=0.744, Precision=0.783, Recall=0.709


In [22]:
roc_auc = roc_auc_score(y_test, preds)
roc_auc

0.8502156211680743

In [23]:
X_test.iloc[2:3]

,id,keyword,location,text,target
773,1119,blew%20up,"Queens, NY",Lmao that light skin guy blew up on Twitter by talking about how ugly he was as a kid..,0


In [24]:
preds = pipeline.predict_proba(X_test.iloc[2:3])[:, 1]
preds

array([0.0871772])

In [26]:
with open('pred_danger.dill', 'wb') as f:
   dill.dump(pipeline, f)